In [ ]:
!pip install tensorflow

In [ ]:
from google.colab import drive, files
import numpy as np
import cv2
import os
import glob
from IPython.display import display, HTML, Image
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Button
from tensorflow.keras.models import load_model

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the base directory
base_dir = "/content/drive/My Drive/MyProject"

# Directory containing the images
images_dir = os.path.join(base_dir, "brain_tumor_dataset")

# Output directory
output_dir = os.path.join(base_dir, "Output")

In [ ]:
# Check if output directory exists, if not, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Adjust contrast and saturation of the image
def adjust_contrast_and_saturation(image, contrast=1.0, saturation=1.0):
    # Adjust contrast
    f = 131 * (contrast + 127) / (127 * (131 - contrast))
    alpha_c = f
    gamma_c = 127 * (1 - f)
    image = cv2.addWeighted(image, alpha_c, image, 0, gamma_c)

    # Convert to HSV, adjust saturation, and convert back to BGR
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv_img[..., 1] = hsv_img[..., 1] * saturation
    image = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return image

In [ ]:
# Model files
model_dir = os.path.join(base_dir, "model_v1")
generator_model_path = os.path.join(model_dir, "generator_epoch_150.h5")
discriminator_model_path = os.path.join(model_dir, "discriminator_epoch_150.h5")

In [ ]:
# Load the generator and discriminator models
generator = load_model(generator_model_path)
discriminator = load_model(discriminator_model_path)

In [ ]:
# Compile the generator model
generator.compile(optimizer='adam', loss='mean_squared_error')

# Compile the discriminator model
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Check if the generator has an optimizer set
generator_compiled = generator.optimizer is not None

# Check if the discriminator has an optimizer set
discriminator_compiled = discriminator.optimizer is not None

print("Is generator compiled?", generator_compiled)
print("Is discriminator compiled?", discriminator_compiled)

Is generator compiled? True
Is discriminator compiled? True


In [ ]:
def colorize_images(num_images=1, brightness=1.0, contrast=1.0, saturation=1.0):
    image_paths = sorted(glob.glob(os.path.join(images_dir, "*.jpg")))

    for image_path in image_paths[:num_images]:
        image = cv2.imread(image_path)
        if image is None:
            print(f"Failed to load image: {image_path}")
            continue

        # Convert to grayscale
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Resize and normalize the grayscale image
        gray_image = cv2.resize(gray_image, (120, 120)) / 255.0

        # Expand dimensions to match the input shape expected by the generator
        gray_image = np.expand_dims(gray_image, axis=-1)
        gray_image = np.expand_dims(gray_image, axis=0)

        # Generate colorized image
        colorized_image = generator.predict(gray_image)

        # Clip values to be within [0, 1] and convert to uint8
        colorized_image = np.clip(colorized_image, 0, 1) * 255
        colorized_image = colorized_image.astype(np.uint8)

        # Resize colorized image to original size
        colorized_image = cv2.resize(colorized_image[0], (image.shape[1], image.shape[0]))

        # Adjust contrast and saturation
        colorized_image = adjust_contrast_and_saturation(colorized_image, contrast, saturation)

        output_path = os.path.join(output_dir, f"adjusted_{os.path.basename(image_path)}")
        cv2.imwrite(output_path, colorized_image)

        # Display the original and adjusted images for comparison
        display(HTML(f"<h3>Original Image: {os.path.basename(image_path)}</h3>"))
        display(Image(filename=image_path))
        display(HTML(f"<h3>Adjusted Image:</h3>"))
        display(Image(filename=output_path))

        # Download button
        download_button = Button(description="Download")
        download_button.on_click(lambda _, path=output_path: files.download(path))
        display(download_button)

In [ ]:
# Define the callback function to colorize images
def on_button_clicked(b):
    colorize_images(num_images=num_images_slider.value, brightness=brightness_slider.value, contrast=contrast_slider.value, saturation=saturation_slider.value)

# Define the callback function to reset sliders to default values
def reset_sliders(b):
    brightness_slider.value = 1.0
    contrast_slider.value = 1.0
    saturation_slider.value = 1.0

In [ ]:
# Slider for selecting the number of images
num_images_slider = widgets.IntSlider(value=1, min=1, max=len(glob.glob(os.path.join(images_dir, "*.jpg"))), step=1, description='Num Images:')

# Sliders for brightness, contrast, and saturation
brightness_slider = widgets.FloatSlider(value=1.0, min=0, max=2.0, step=0.1, description='Brightness')
contrast_slider = widgets.FloatSlider(value=1.0, min=0.5, max=3.0, step=0.1, description='Contrast')
saturation_slider = widgets.FloatSlider(value=1.0, min=0, max=2.0, step=0.1, description='Saturation')

In [ ]:
# Button to trigger colorization
button = widgets.Button(description="Colorize Images")

# Reset button
reset_button = widgets.Button(description="Reset")

In [ ]:
# Attach the functions to the button click events
button.on_click(on_button_clicked)
reset_button.on_click(reset_sliders)

# Layout the widgets vertically
widgets_layout = VBox([num_images_slider, brightness_slider, contrast_slider, saturation_slider, HBox([button, reset_button])])

In [ ]:
# Display the widgets
display(widgets_layout)